In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
from IPython.display import Image, display
from wordcloud import WordCloud
import random
import re

# EXTRACT AND PRE-PROCESSING DATA

## Product dataframe 

In [ ]:
df_product = pd.read_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getdata\combined_data.csv')
df_product

In [ ]:
df_product.info()

In [ ]:
df_product.duplicated().sum()

In [ ]:
df_product = df_product.drop_duplicates()

In [ ]:
df_product = df_product.drop(columns=['countReviews'])
df_product

In [ ]:
df_product.loc[:, 'discounts'] = df_product['discounts'].apply(lambda x: unicodedata.normalize('NFC', str(x)) if pd.notna(x) else x)

In [ ]:
df_product['discounts'] = df_product['discounts'].str.extract(r'(\d+)').astype(float) / 100
df_product

In [ ]:
df_product.loc[:, 'discounts'] = df_product['discounts'].fillna(0)

In [ ]:
df_product['discounts'].median()

In [ ]:
df_product.loc[df_product['discounts'] >= 1, 'discounts'] = df_product['discounts'].median()
df_product['discounts'].describe()

In [ ]:
df_product.loc[:, 'img'] = 'C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/' + df_product.index.astype(str) + '.jpg'
df_product

In [ ]:
for path in df_product.loc[2790:2795, 'img']:
    display(Image(filename=path))

In [ ]:
df_product.loc[:, 'titles'] = df_product['titles'].apply(lambda x: unicodedata.normalize('NFC', x))

In [ ]:
len(df_product[df_product['prices'] <= 0])

In [ ]:
df_product[df_product['prices'] <= 0]

In [ ]:
df_product = df_product.drop([1208])

In [ ]:
len(df_product[df_product['countSales'] < 0])

### Classifying products from images and titles ---> *failed*

In [ ]:
csv_path = 'C:/Users/ASUS/Desktop/T/ĐAN_KLTN/kaggle_clothes_train/images.csv'
img_dir = 'C:/Users/ASUS/Desktop/T/ĐAN_KLTN/kaggle_clothes_train/images_compressed'

df_train_img = pd.read_csv(csv_path)
df_train_img

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

IMG_SIZE = (64, 64)

X_train, y_train = [], []
for _, row in df_train_img.iterrows():
    img_path = os.path.join(img_dir, row['image'] + '.jpg')
    try:
        img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
        img_array = np.array(img).flatten()
        X_train.append(img_array)
        y_train.append(row['label'])
    except:
        print(f"Lỗi đọc ảnh train: {img_path}")

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train_enc)

X_predict = []
valid_img_names = []
failed_imgs = []

In [ ]:
for _, row in df_product.iterrows():
    img_path = os.path.join(img_dir, row['img'])
    try:
        img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
        img_array = np.array(img).flatten()
        X_predict.append(img_array)
        valid_img_names.append(row['img'])
    except:
        print(f"Lỗi đọc ảnh predict: {img_path}")
        failed_imgs.append(row['img'])

In [ ]:
y_pred_enc = clf.predict(X_predict)
y_pred_labels = le.inverse_transform(y_pred_enc)

df_pred = pd.DataFrame({'img': valid_img_names, 'category': y_pred_labels})
df_product = df_product.merge(df_pred, on='img', how='left')

print(df_product[['img', 'category']].head())

In [ ]:
df_product['category'].value_counts()

In [ ]:
df_product.loc[:, 'category'] = df_product['category'].fillna('Not sure')

In [ ]:
df_known = df_product[df_product['category'] != 'Not sure']
df_unknown = df_product[df_product['category'] == 'Not sure']

In [ ]:
df_known

In [ ]:
df_unknown

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
X_known = vectorizer.fit_transform(df_known['titles'].astype(str))
X_unknown = vectorizer.transform(df_unknown['titles'].astype(str))

le_nlp = LabelEncoder()
y_known = le_nlp.fit_transform(df_known['category'])

In [ ]:
clf_nlp = RandomForestClassifier(n_estimators=100, random_state=42)
clf_nlp.fit(X_known, y_known)
y_pred_nlp = clf_nlp.predict(X_unknown)
predicted_labels = le_nlp.inverse_transform(y_pred_nlp)
df_product.loc[df_product['category'] == 'Not sure', 'category'] = predicted_labels

In [ ]:
df_product['category'].value_counts()

In [ ]:
def show_images_with_categories(df, img_dir, n=12):
    df = df.reset_index(drop=True)
    total = min(n, len(df))
    cols = 4
    rows = (total // cols) + (total % cols > 0)
    
    plt.figure(figsize=(4 * cols, 4 * rows))
    
    for i in range(total):
        img_name = df.loc[i, 'img']
        category = df.loc[i, 'category']
        img_path = os.path.join(img_dir, img_name)
        
        try:
            img = Image.open(img_path).convert('RGB')
            plt.subplot(rows, cols, i + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(category, fontsize=10)
        except:
            print(f"Lỗi khi mở ảnh: {img_path}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
show_images_with_categories(df_product, img_dir, n=100)

### Classifying products from titles (manual)

In [ ]:
category_keywords = {
    'T-Shirt': ['t-shirt', 'tee', 'áo thun'],
    'Pants': ['pants', 'quần dài', 'trousers', 'jeans', 'quần'],
    'Longsleeve': ['longsleeve', 'dài tay', 'áo tay dài', 'áo dài'],
    'Shoes': ['giày', 'shoes', 'sneaker', 'loafers', 'boot', 'dép', 'dép lê', 'sục', 'boost', 'bốt', 'high heels', 'guốc', 'cao gót', 'sandal', 'xăng đan'],
    'Dress': ['váy', 'dress', 'đầm', 'đầm xòe'],
    'Shirt': ['shirt', 'sơ mi', 'áo sơ mi', 'áo'],
    'Hoodie': ['hoodie'],
    'Outwear': ['áo khoác', 'khoác', 'jacket', 'coat', 'outerwear'],
    'Shorts': ['short', 'quần short', 'quần đùi'],
    'Body': ['body', 'bodysuit'],
    'Hat': ['hat', 'mũ', 'nón', 'rộng vành', 'lưỡi trai', 'tóc'],
    'Undershirt': ['undershirt', 'áo lót', 'áo trong', 'bra'],
    'Skirt': ['skirt', 'chân váy', 'váy tennis', 'váy xòe', 'xòe'],
    'Blazer': ['blazer'],
    'Set': ['set', 'sét', 'đồ ngủ', 'pijama', 'đồ bộ', 'bộ đồ', 'đồ lam', 'bộ', 'pháp phục', 'jump', 'đồng phục']
}

In [ ]:
import re

def classify_from_title(title):
    title = re.sub(r'[^\w\s]', ' ', title)
    title = re.sub(r'[\U00010000-\U0010ffff]', '', title)  # remove emojis
    title = re.sub(r'\s+', ' ', title).strip()
    
    title_lower = title.lower()
    categories_found = []
    
    for category, keywords in category_keywords.items():
        if any(keyword in title_lower for keyword in keywords):
            categories_found.append(category)
            
        if len(categories_found) == 3: # maximun 3 categories
            break
            
    return categories_found

In [ ]:
df_product['category'] = df_product.apply(lambda row: classify_from_title(row['titles']), axis=1)
df_product['category'].value_counts()

In [ ]:
df_product[df_product['category'].apply(lambda x: len(x) == 0)]

In [ ]:
df_product['category'] = df_product['category'].apply(lambda x: ['Not sure'] if isinstance(x, list) and len(x) == 0 else x)

### Overview

In [ ]:
df_product.info()

In [ ]:
df_product

## Product's comments dataframe 

In [ ]:
df_cmt = pd.read_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\getcomment\combined_data.csv')
df_cmt

In [ ]:
df_cmt.info()

In [ ]:
df_cmt = df_cmt.drop(columns=['like_count'])
df_cmt = df_cmt.drop(columns=['stt'])
df_cmt = df_cmt.drop(columns=['name_comment'])
df_cmt

In [ ]:
df_cmt['star_count'] = pd.to_numeric(df_cmt['star_count'], errors='coerce').fillna(0).astype(int)

In [ ]:
df_cmt[(df_cmt['star_count'] < 0) | (df_cmt['star_count'] > 5)]

In [ ]:
df_cmt['content_comment'].isna().sum()

In [ ]:
df_cmt = df_cmt.dropna(subset=['content_comment'])

In [ ]:
df_cmt.loc[:, 'content_comment'] = df_cmt['content_comment'].apply(lambda x: unicodedata.normalize('NFC', x) if isinstance(x, str) else x)
df_cmt

In [ ]:
df_cmt.info()

# VISUALIZE ATTRIBUTES IN DATAFRAMES

## Product dataframe 

In [ ]:
df_product.describe()

In [ ]:
df_product['prices'].describe().apply(lambda x: format(x, ',.0f'))

In [ ]:
price_bins = [0, 100000, 500000, 1000000, float('inf')]
price_labels = ['0-100k', '100k-500k', '500k-1M', 'Above 1M']
df_product['price_range'] = pd.cut(df_product['prices'], bins=price_bins, labels=price_labels, right=False)

In [ ]:
price_range_count = df_product['price_range'].value_counts()
price_range_count

In [ ]:
price_range_percentage = (price_range_count / price_range_count.sum()) * 100
price_range_percentage

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].pie(
    price_range_percentage,
    labels=[f'{label} ({pct:.1f}%)' for label, pct in zip(price_range_percentage.index, price_range_percentage)],
    startangle=140,
    colors=plt.cm.Pastel1.colors,
    autopct='%1.1f%%'
)
axes[0].set_title('Product Price Range - Pie Chart')
axes[0].axis('equal')

sns.countplot(ax=axes[1], x='price_range', data=df_product, palette='Pastel1', order=price_labels)
axes[1].set_title('Product Price Range - Bar Chart')
axes[1].set_xlabel('Price Range')
axes[1].set_ylabel('Number of Products')
plt.tight_layout()
plt.show()

In [ ]:
df_product['discounts'].describe()

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(x=df_product['discounts'], color='skyblue')
plt.title('Boxplot of Product Discounts')
plt.xlabel('Discount (%)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_product['discounts'], kde=True, color='lightgreen', bins=30)
plt.title('Discount Distribution')
plt.xlabel('Discount (%)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
len(df_product[df_product['discounts'] == 0])

In [ ]:
df_product['countSales'].describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_product['countSales'], kde=True, color='orange', bins=30)
plt.title('Product Sales Distribution')
plt.xlabel('Number of Sales')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='prices', y='countSales', hue='price_range', data=df_product, palette='viridis', alpha=0.6)
plt.title('Prices vs. Sales Count', fontsize=16)
plt.xlabel('Price', fontsize=12)
plt.ylabel('Sales Count', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
price_above500k_sales_above500 = df_product[(df_product['prices'] > 500000) & (df_product['countSales'] > 500)]
price_above500k_sales_above500

In [ ]:
category_counts = df_product.explode('category')['category'].value_counts()
category_counts

In [ ]:
unique_categories = df_product.explode('category')['category'].unique()
unique_categories

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(data=df_product.explode('category').reset_index(drop=True), x='category', y='discounts', palette='Set3')
plt.title('Distribution of Discounts by Product Category')
plt.xlabel('Category')
plt.ylabel('Discounts')
plt.tight_layout()
plt.show()

In [ ]:
df_product.explode('category').groupby('category')['discounts'].describe()

In [ ]:
discounted_per_category = df_product.explode('category')[df_product.explode('category')['discounts'] > 0.0].groupby('category').size()
discounted_per_category

In [ ]:
discount_per_category_ratio = (discounted_per_category / category_counts).fillna(0).sort_values(ascending=False) * 100
discount_per_category_ratio

In [ ]:
no_discount_per_category_ratio = 100 - discount_per_category_ratio
stacked_ratio_df = pd.DataFrame({'Discounted (%)': discount_per_category_ratio, 'Not Discounted (%)': no_discount_per_category_ratio}).sort_values('Discounted (%)')
stacked_ratio_df.plot(kind='barh', stacked=True, figsize=(10, 7), color=['#4CAF50', '#c1e3c1'])

plt.xlabel('Percentage of Products')
plt.title('Percentage of Discounted vs Non-Discounted Products by Category')
plt.legend(title='Discount Status', loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
import math

n_categories = len(unique_categories)
n_cols = 2
n_rows = math.ceil(n_categories / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6 * n_rows))

for i, category in enumerate(unique_categories):
    category_data = df_product[df_product['category'].apply(lambda x: category in x)]
    row = i // n_cols
    col = i % n_cols
    sns.countplot(ax=axes[row, col], x='price_range', data=category_data, palette='Pastel1')
    axes[row, col].set_title(f'Price Range Distribution for {category}')
    axes[row, col].set_xlabel('Price Range')
    axes[row, col].set_ylabel('Number of Products')
    
if n_categories % n_cols != 0:
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
price_range_category = df_product.explode('category').groupby(['category', 'price_range']).size().reset_index(name='count')
price_range_category

In [ ]:
pivot_df = price_range_category.pivot(index='category', columns='price_range', values='count').fillna(0)
pivot_df = pivot_df.loc[pivot_df.sum(axis=1).sort_values(ascending=False).index]

pivot_df.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='Pastel1', edgecolor='black')
plt.title("Product Distribution by Category and Price Range", fontsize=16)
plt.xlabel("Product Category")
plt.ylabel("Number of Products")
plt.xticks(rotation=45)
plt.legend(title="Price Range")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_product, x='discounts', y='countSales', alpha=0.6)
plt.title('Relationship between Discounts and Sales Count')
plt.xlabel('Discount (as a fraction, e.g., 0.4 = 40%)')
plt.ylabel('Sales Count')
plt.tight_layout()
plt.show()

In [ ]:
df_product['discounted'] = df_product['discounts'] > 0
avg_sales_overall = df_product.groupby('discounted')['countSales'].mean()
avg_sales_by_category = df_product.explode('category').groupby(['category', 'discounted'])['countSales'].mean().unstack()

fig, axes = plt.subplots(2, 1, figsize=(12, 10))
avg_sales_overall.plot(kind='bar', color=['#a3c9f1', '#2166ac'], ax=axes[0])
axes[0].set_title('Average Units Sold: Discounted vs Non-Discounted')
axes[0].set_ylabel('Average Units Sold')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['No Discount', 'Discount'], rotation=0)

avg_sales_by_category.plot(kind='bar', ax=axes[1], color=['#a3c9f1', '#2166ac'])
axes[1].set_title('Average Sales with and without Discount')
axes[1].set_ylabel('Average Units Sold')
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud
titles = ' '.join(df_product['titles'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(titles)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bicubic')
plt.axis('off')
plt.show()

In [ ]:
import unicodedata
import nltk
from nltk.tokenize import word_tokenize

def generate_ngrams(string, n):
    string = unicodedata.normalize('NFC', string)
    tokens = word_tokenize(string, language='english')
    tokens = [token.lower() for token in tokens]
    return [' '.join(tokens[i:i + n]) for i in range(len(tokens) - n + 1)]

In [ ]:
from collections import Counter

_1gram = generate_ngrams(titles, 1)
word_counts = Counter(_1gram)
word_counts.most_common(20)

In [ ]:
_2grams = generate_ngrams(titles, 2)
_2words_counts = Counter(_2grams)
_2words_counts.most_common(20)

## Product's comments dataframe 

In [ ]:
df_cmt.info()

In [ ]:
df_cmt.describe()

In [ ]:
star_counts = df_cmt['star_count'].value_counts()
star_counts

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='star_count', data=df_cmt, palette='Accent')
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height}', 
                (p.get_x() + p.get_width() / 2., height), 
                ha='center', va='bottom', fontsize=10)  

plt.title('Star Rating Distribution in Comments')
plt.xlabel('Star Rating')
plt.ylabel('Number of Comments')
plt.show()

In [ ]:
df = pd.merge(df_product, df_cmt, left_on='links', right_on='Link', how='inner')
df

In [ ]:
df_cmt_grouped = df_cmt.groupby('Link').agg({
    'content_comment': list,
    'skuInfo_comment': list,
    'star_count': list
}).reset_index()
df_merged = pd.merge(df_product, df_cmt_grouped, how='left', left_on='links', right_on='Link')
df_merged = df_merged.drop(columns=['Link'])
df_merged

In [ ]:
df_merged['avg_star'] = df_merged['star_count'].apply(lambda stars: sum(stars) / len(stars) if isinstance(stars, list) and len(stars) > 0 else None)
df_merged

In [ ]:
df_merged['avg_star'].describe()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_merged, y='avg_star', color='violet')
plt.title('Distribution of Average Star Ratings')
plt.ylabel('Average Star Rating')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='star_count', y='countSales', data=df, palette='Set3')
plt.title('Sales Distribution by Star Rating')
plt.xlabel('Star Rating')
plt.ylabel('Number of Sales')
plt.show()

In [ ]:
df_merged['low_rating'] = df_merged['star_count'].apply(lambda stars: any(s <= 3 for s in stars if pd.notnull(s)) if isinstance(stars, list) else False)
df_merged['low_rating'].value_counts()

In [ ]:
df_lowrating_category = df_merged[df_merged['low_rating'] == True]
df_lowrating_category

In [ ]:
df_lowrating_category_exploded = df_lowrating_category.copy()
df_lowrating_category_exploded = df_lowrating_category_exploded.explode('category')
df_lowrating_category_exploded

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_lowrating_category_exploded, y='category', order=df_lowrating_category_exploded['category'].value_counts().index, palette='viridis')
plt.title('Distribution of Low-Rated Products (Rating ≤ 3) by Category')
plt.xlabel('Number of Products')
plt.ylabel('Product Category')
plt.tight_layout()
plt.show()

In [ ]:
category_counts = df_lowrating_category_exploded['category'].value_counts().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='Set3')
plt.title('Number of Low-Rated Products (≤ 3 stars) by Category')
plt.xlabel('Product Category')
plt.ylabel('Number of Products')
plt.tight_layout()
plt.show()

In [ ]:
df_price_star = df_merged.groupby('price_range')['avg_star'].mean().reset_index()
df_price_star['price_range'] = pd.Categorical(df_price_star['price_range'], categories=sorted(df_price_star['price_range'].unique(), key=lambda x: str(x)))
df_price_star = df_price_star.sort_values('price_range')

plt.figure(figsize=(10, 6))
sns.barplot(data=df_price_star, x='price_range', y='avg_star', palette='coolwarm')
plt.title('Average Star Rating by Price Range', fontsize=14)
plt.xlabel('Price Range')
plt.ylabel('Average Star Rating')
plt.tight_layout()
plt.show()

In [ ]:
df_price_star

In [ ]:
records = []

for _, row in df_lowrating_category_exploded.iterrows():
    category = row['category']
    stars = row['star_count']
    if isinstance(stars, list):
        for s in stars:
            if pd.notnull(s) and s in [1, 2, 3]:
                records.append((category, s))

df_counts = pd.DataFrame(records, columns=['category', 'star'])
df_plot = df_counts.groupby(['category', 'star']).size().unstack(fill_value=0)
df_plot = df_plot[[1, 2, 3]]

df_plot.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='Paired')
plt.title('Stacked Bar Chart of Low Star Ratings by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Number of Ratings')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Star Rating')
plt.tight_layout()
plt.show()

In [ ]:
df_plot

In [ ]:
text = ' '.join(df_cmt[df_cmt['star_count'] <= 3]['content_comment'])
wordcloud = WordCloud(width=1000, height=500, background_color='white', colormap='plasma', font_path='arial.ttf').generate(text)
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('WordCloud of Low-Rated Product Comments', fontsize=16)
plt.show()

In [ ]:
_1gram = generate_ngrams(text, 1)
word_counts = Counter(_1gram)
word_counts.most_common(10)

In [ ]:
_2gram = generate_ngrams(text, 2)
word_counts = Counter(_2gram)
word_counts.most_common(10)

In [ ]:
_3gram = generate_ngrams(text, 3)
word_counts = Counter(_3gram)
word_counts.most_common(10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

bad_comments = df_cmt[df_cmt['star_count'] <= 3]['content_comment'].dropna().astype(str)
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_tfidf = vectorizer.fit_transform(bad_comments)

tfidf_scores = np.asarray(X_tfidf.mean(axis=0)).ravel()
top_words = sorted(zip(vectorizer.get_feature_names_out(), tfidf_scores), key=lambda x: x[1], reverse=True)

for word, score in top_words[:10]:
    print(f"{word}: {score:.4f}")

In [ ]:
top_10 = top_words[:10]
words, scores = zip(*top_10)

plt.figure(figsize=(10, 6))
plt.barh(words[::-1], scores[::-1], color='skyblue')
plt.xlabel('TF-IDF Score')
plt.title('Top 10 TF-IDF Words in Low-Rated Comments')
plt.tight_layout()
plt.show()

In [ ]:
text = ' '.join(df_cmt['content_comment'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Comments')
plt.show()

In [ ]:
_1gram = generate_ngrams(text, 1)
word_counts = Counter(_1gram)
word_counts.most_common(20)

In [ ]:
_2grams = generate_ngrams(text, 2)
_2words_counts = Counter(_2grams)
_2words_counts.most_common(20)

In [ ]:
_3grams = generate_ngrams(text, 3)
_3words_counts = Counter(_3grams)
_3words_counts.most_common(20)

In [ ]:
comments = df_cmt['content_comment'].dropna().astype(str)
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_tfidf = vectorizer.fit_transform(comments)

tfidf_scores = np.asarray(X_tfidf.mean(axis=0)).ravel()
top_words = sorted(zip(vectorizer.get_feature_names_out(), tfidf_scores), key=lambda x: x[1], reverse=True)

In [ ]:
for word, score in top_words[:10]:
    print(f"{word}: {score:.4f}")

In [ ]:
top_10 = top_words[:10]
words, scores = zip(*top_10)

plt.figure(figsize=(10, 6))
plt.barh(words[::-1], scores[::-1], color='plum')
plt.xlabel('TF-IDF Score')
plt.title('Top 10 TF-IDF Words in Comments')
plt.tight_layout()
plt.show()

# GENERATE DATA

In [ ]:
df_product.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\backup\df_product.csv')
df_cmt.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\backup\df_cmt.csv')

In [3]:
df_product = pd.read_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\backup\df_product.csv')
df_cmt = pd.read_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\backup\df_cmt.csv')

In [4]:
df_product

,Unnamed: 0,titles,links,prices,discounts,countSales,img,category,price_range,discounted
0,0,Dép Lê Thoáng Khí Đục Lỗ Bít Mũi Mùa Hè Đi Ra ...,https://www.lazada.vn/products/dep-le-thoang-k...,95000,0.40,78,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/0.jpg,"['Shoes', 'Hat']",0-100k,True
1,1,Dép Đế Xuồng Đế Dày Thời Trang Mùa Hè Cho Nữ Đ...,https://www.lazada.vn/products/dep-de-xuong-de...,62000,0.87,6,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/1.jpg,['Shoes'],0-100k,True
2,2,SPICYG Darlingism Đầm Maxistyle Dài Thời Trang...,https://www.lazada.vn/products/spicyg-darlingi...,171071,0.58,22,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/2.jpg,"['Longsleeve', 'Dress', 'Shirt']",100k-500k,True
3,3,ISARO Giày thể thao vải lưới thời trang 2024 d...,https://www.lazada.vn/products/isaro-giay-the-...,45000,0.50,31,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/3.jpg,['Shoes'],0-100k,True
4,4,"Dép nơ thời trang nữ, gót vuông 3 phân phù hợp...",https://www.lazada.vn/products/dep-no-thoi-tra...,120000,0.00,51,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/4.jpg,['Shoes'],100k-500k,False
...,...,...,...,...,...,...,...,...,...,...
7155,7155,Bộ tập pickleball áo polo PL chân váy xoè xếp ...,https://www.lazada.vn/products/bo-tap-pickleba...,278000,0.15,6,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/715...,"['Dress', 'Shirt', 'Skirt']",100k-500k,True
7156,7156,Đồ tập gym yoga nữ set bộ áo crop ngắn tay kèm...,https://www.lazada.vn/products/do-tap-gym-yoga...,181000,0.47,521,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/715...,"['Pants', 'Shirt', 'Set']",100k-500k,True
7157,7157,mẫu đồ lam đi chùɑ #bộ bà lai gấm phối lụa-phá...,https://www.lazada.vn/products/mau-do-lam-di-c...,209000,0.00,27,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/716...,['Set'],100k-500k,False
7158,7158,[HCM] Đồ bộ lanh (tole) trung niên SHOP5SAO ta...,https://www.lazada.vn/products/hcm-do-bo-lanh-...,140600,0.24,6,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/716...,"['Pants', 'Set']",100k-500k,True


In [5]:
df_cmt

,Unnamed: 0,content_comment,skuInfo_comment,star_count,Link
0,0,"mang êm chân lắm, ai thích mang ôm chân lấy n...","Nhóm màu:Nâu, Kích cỡ:40-41",5,https://www.lazada.vn/products/dep-le-thoang-k...
1,1,"Bền và chắc chắn, Khuyến khích trò chơi tưởng ...","Nhóm màu:Nâu, Kích cỡ:38-39",5,https://www.lazada.vn/products/dep-le-thoang-k...
2,2,hàng tốt,"Nhóm màu:Màu cam, Kích cỡ:36-37",5,https://www.lazada.vn/products/dep-le-thoang-k...
3,3,"Dép mang khg thoải mái,vì vô sát cổ chân quá,h...","Nhóm màu:Đen, Kích cỡ:36-37",3,https://www.lazada.vn/products/dep-le-thoang-k...
4,4,Nhẹ nhàng thoải mái độ cao vừa giúp chân sạch ...,"Nhóm màu:Đen, Kích cỡ:36-37",5,https://www.lazada.vn/products/dep-le-thoang-k...
...,...,...,...,...,...
126918,127621,"Hoàn hảo cho mùa hè, Họa tiết hoa phong cách, ...","Nhóm Màu:Trắng, Kích cỡ:Int:XXL",5,https://www.lazada.vn/products/set-do-nu-kem-c...
126919,127622,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...","Nhóm Màu:Xanh, Kích cỡ:Int:XXL",5,https://www.lazada.vn/products/set-do-nu-kem-c...
126920,127623,"Đa năng cho trang phục văn phòng, Họa tiết hoa...","Nhóm Màu:Xanh, Kích cỡ:Int:S",5,https://www.lazada.vn/products/set-do-nu-kem-c...
126921,127624,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...","Nhóm Màu:Xanh, Kích cỡ:Int:M",5,https://www.lazada.vn/products/set-do-nu-kem-c...


In [6]:
def bo_dau(chuoi):
    chuoi = chuoi.lower()
    chuoi = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', chuoi)
    chuoi = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', chuoi)
    chuoi = re.sub(r'[ìíịỉĩ]', 'i', chuoi)
    chuoi = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', chuoi)
    chuoi = re.sub(r'[ùúụủũưừứựửữ]', 'u', chuoi)
    chuoi = re.sub(r'[ỳýỵỷỹ]', 'y', chuoi)
    chuoi = re.sub(r'[đ]', 'd', chuoi)
    return chuoi

In [7]:
permissions = [
    {"permission_id": "MANAGE_USERS", "description": "Quản lý người dùng"},
    {"permission_id": "MANAGE_PRODUCTS", "description": "Quản lý sản phẩm"},
    {"permission_id": "MANAGE_ORDERS", "description": "Xử lý đơn hàng"},
    {"permission_id": "MANAGE_REVIEWS", "description": "Quản lý đánh giá"},
    {"permission_id": "MANAGE_CATEGORIES", "description": "Quản lý danh mục sản phẩm"},
    {"permission_id": "VIEW_REPORT", "description": "Xem các báo cáo phân tích/thống kê"},
    {"permission_id": "MANAGE_PERMISSIONS", "description": "Phân quyền quản trị"},
    {"permission_id": "VIEW_LOGS", "description": "Xem lịch sử hoạt động"},
]
df_permission = pd.DataFrame(permissions)
df_permission

,permission_id,description
0,MANAGE_USERS,Quản lý người dùng
1,MANAGE_PRODUCTS,Quản lý sản phẩm
2,MANAGE_ORDERS,Xử lý đơn hàng
3,MANAGE_REVIEWS,Quản lý đánh giá
4,MANAGE_CATEGORIES,Quản lý danh mục sản phẩm
5,VIEW_REPORT,Xem các báo cáo phân tích/thống kê
6,MANAGE_PERMISSIONS,Phân quyền quản trị
7,VIEW_LOGS,Xem lịch sử hoạt động


In [8]:
df_permission.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\permissions.csv', index=False)

In [9]:
ho = ['Nguyễn', 'Trần', 'Lê', 'Phạm', 'Hoàng', 'Huỳnh', 'Phan', 'Vũ', 'Võ', 
      'Đặng', 'Bùi', 'Đỗ', 'Hồ', 'Ngô', 'Dương', 'Lý', 'Đào', 'Đinh', 'Mai', 'Trương']

ten_dem = ['Văn', 'Thị', 'Hữu', 'Đức', 'Minh', 'Thanh', 'Công', 'Thành', 'Kim', 'Ngọc']
ten = ['An', 'Bình', 'Chi', 'Dũng', 'Giang', 'Hương', 'Khánh', 'Linh', 'Nam', 'Phương']

admins = []
for i in range(1, 101):
    random_ho = random.choice(ho)
    random_dem = random.choice(ten_dem)
    random_ten = random.choice(ten)

    email_ho = bo_dau(random_ho).replace(' ', '')
    email_ten = bo_dau(random_ten).replace(' ', '')
    
    admins.append({
        'admin_id': f'C{i:03d}',
        'last_name': random_ho,
        'first_name': f'{random_dem} {random_ten}',
        'email': f'{email_ho}.{email_ten}{i}@gmail.com',
        'password': 'P@ssw0rd123'
    })

df_admin = pd.DataFrame(admins)
df_admin

,admin_id,last_name,first_name,email,password
0,C001,Trần,Công An,tran.an1@gmail.com,P@ssw0rd123
1,C002,Võ,Thanh Dũng,vo.dung2@gmail.com,P@ssw0rd123
2,C003,Dương,Thanh Bình,duong.binh3@gmail.com,P@ssw0rd123
3,C004,Hồ,Ngọc Chi,ho.chi4@gmail.com,P@ssw0rd123
4,C005,Hoàng,Công Khánh,hoang.khanh5@gmail.com,P@ssw0rd123
...,...,...,...,...,...
95,C096,Hoàng,Công An,hoang.an96@gmail.com,P@ssw0rd123
96,C097,Đỗ,Thành Bình,do.binh97@gmail.com,P@ssw0rd123
97,C098,Bùi,Đức Nam,bui.nam98@gmail.com,P@ssw0rd123
98,C099,Bùi,Minh Giang,bui.giang99@gmail.com,P@ssw0rd123


In [10]:
df_admin.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\admins.csv', index=False)

In [11]:
admin_ids = df_admin['admin_id'].tolist()
permission_ids = df_permission['permission_id'].tolist()

admin_has_permissions = []

for admin_id in admin_ids:
    num_permissions = random.randint(1, 4)
    assigned_permissions = random.sample(permission_ids, num_permissions)
    for permission_id in assigned_permissions:
        admin_has_permissions.append({
            "permission_id": permission_id,
            "admin_id": admin_id
        })

df_adminhaspermissions = pd.DataFrame(admin_has_permissions)
df_adminhaspermissions

,permission_id,admin_id
0,MANAGE_ORDERS,C001
1,VIEW_REPORT,C001
2,VIEW_REPORT,C002
3,MANAGE_ORDERS,C003
4,VIEW_LOGS,C004
...,...,...
264,VIEW_LOGS,C099
265,MANAGE_PRODUCTS,C100
266,MANAGE_CATEGORIES,C100
267,MANAGE_ORDERS,C100


In [12]:
df_adminhaspermissions.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\adminhaspermissions.csv', index=False)

In [13]:
categories = [
    {"category_id": "SHIRT", "description": "Áo sơ mi"},
    {"category_id": "DRESS", "description": "Đầm/Váy liền"},
    {"category_id": "SET", "description": "Bộ trang phục"},
    {"category_id": "PANTS", "description": "Quần dài"},
    {"category_id": "LONGSLEEVE", "description": "Áo tay dài"},
    {"category_id": "SKIRT", "description": "Chân váy"},
    {"category_id": "SHOES", "description": "Giày dép"},
    {"category_id": "TSHIRT", "description": "Áo thun"},
    {"category_id": "SHORTS", "description": "Quần ngắn"},
    {"category_id": "HAT", "description": "Mũ/nón"},
    {"category_id": "BODY", "description": "Đồ body/áo liền quần"},
    {"category_id": "OUTWEAR", "description": "Áo khoác ngoài"},
    {"category_id": "NOT_SURE", "description": "Chưa phân loại"},
    {"category_id": "UNDERSHIRT", "description": "Áo lót/áo ba lỗ"},
    {"category_id": "HOODIE", "description": "Áo hoodie/áo nỉ có mũ"},
    {"category_id": "BLAZER", "description": "Áo blazer/áo vest"}
]

df_category = pd.DataFrame(categories)
df_category

,category_id,description
0,SHIRT,Áo sơ mi
1,DRESS,Đầm/Váy liền
2,SET,Bộ trang phục
3,PANTS,Quần dài
4,LONGSLEEVE,Áo tay dài
5,SKIRT,Chân váy
6,SHOES,Giày dép
7,TSHIRT,Áo thun
8,SHORTS,Quần ngắn
9,HAT,Mũ/nón


In [14]:
df_category.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\categories.csv', index=False)

In [15]:
df_payment = pd.DataFrame([
    {"payment_id": "COD", "description": "thanh toán khi nhận hàng"},
    {"payment_id": "CREDIT_CARD", "description": "thẻ tín dụng"},
    {"payment_id": "BANK_TRANSFER", "description": "chuyển khoản ngân hàng"},
    {"payment_id": "E_WALLET", "description": "ví điện tử"},
])
df_payment

,payment_id,description
0,COD,thanh toán khi nhận hàng
1,CREDIT_CARD,thẻ tín dụng
2,BANK_TRANSFER,chuyển khoản ngân hàng
3,E_WALLET,ví điện tử


In [16]:
df_payment.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\payments.csv', index=False)

In [17]:
df_orderstatus = pd.DataFrame([
    {"status_id": "PENDING", "description": "Chờ xác nhận"},
    {"status_id": "CONFIRMED", "description": "Đã xác nhận"},
    {"status_id": "PROCESSING", "description": "Đang xử lý"},
    {"status_id": "SHIPPED", "description": "Đã giao hàng"},
    {"status_id": "CANCELLED", "description": "Đã hủy đơn"},
])
df_orderstatus

,status_id,description
0,PENDING,Chờ xác nhận
1,CONFIRMED,Đã xác nhận
2,PROCESSING,Đang xử lý
3,SHIPPED,Đã gaio hàng
4,CANCELLED,Đã hủy đơn


In [18]:
df_orderstatus.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\orderstatuses.csv', index=False)

In [19]:
ho = ['Nguyễn', 'Trần', 'Lê', 'Phạm', 'Hoàng', 'Huỳnh', 'Phan', 'Vũ', 'Võ', 
      'Đặng', 'Bùi', 'Đỗ', 'Hồ', 'Ngô', 'Dương', 'Lý', 'Đào', 'Đinh', 'Mai', 'Trương']

ten_dem = ['Văn', 'Thị', 'Hữu', 'Đức', 'Minh', 'Thanh', 'Công', 'Thành', 'Kim', 'Ngọc']
ten = ['An', 'Bình', 'Chi', 'Dũng', 'Giang', 'Hương', 'Khánh', 'Linh', 'Nam', 'Phương']

customers = []
for i in range(1, 101):
    random_ho = random.choice(ho)
    random_dem = random.choice(ten_dem)
    random_ten = random.choice(ten)

    email_ho = bo_dau(random_ho).replace(' ', '')
    email_ten = bo_dau(random_ten).replace(' ', '')
    
    customers.append({
        'customer_id': f'C{i:03d}',
        'last_name': random_ho,
        'first_name': f'{random_dem} {random_ten}',
        'email': f'{email_ho}.{email_ten}{i}@gmail.com',
        'password': 'Abcxyz@123',
        'phone_number': f'098765432{i%10}',
        'address': f'{i+1} Đường số {i}, Quận {i%5+1}, TP.HCM'
    })

df_customer = pd.DataFrame(customers)
df_customer

,customer_id,last_name,first_name,email,password,phone_number,address
0,C001,Hồ,Hữu Giang,ho.giang1@gmail.com,Abcxyz@123,0987654321,"2 Đường số 1, Quận 2, TP.HCM"
1,C002,Đinh,Thành Chi,dinh.chi2@gmail.com,Abcxyz@123,0987654322,"3 Đường số 2, Quận 3, TP.HCM"
2,C003,Phạm,Ngọc Chi,pham.chi3@gmail.com,Abcxyz@123,0987654323,"4 Đường số 3, Quận 4, TP.HCM"
3,C004,Lý,Văn Nam,ly.nam4@gmail.com,Abcxyz@123,0987654324,"5 Đường số 4, Quận 5, TP.HCM"
4,C005,Huỳnh,Kim Chi,huynh.chi5@gmail.com,Abcxyz@123,0987654325,"6 Đường số 5, Quận 1, TP.HCM"
...,...,...,...,...,...,...,...
95,C096,Vũ,Đức Hương,vu.huong96@gmail.com,Abcxyz@123,0987654326,"97 Đường số 96, Quận 2, TP.HCM"
96,C097,Phan,Ngọc Chi,phan.chi97@gmail.com,Abcxyz@123,0987654327,"98 Đường số 97, Quận 3, TP.HCM"
97,C098,Trần,Văn Linh,tran.linh98@gmail.com,Abcxyz@123,0987654328,"99 Đường số 98, Quận 4, TP.HCM"
98,C099,Phan,Văn Chi,phan.chi99@gmail.com,Abcxyz@123,0987654329,"100 Đường số 99, Quận 5, TP.HCM"


In [20]:
df_customer.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\customers.csv', index=False)

In [21]:
df_product_backup = pd.DataFrame({
    'product_id': [f'P{i:04d}' for i in range(1, len(df_product)+1)],
    'name': df_product['titles'].values,
    'description': np.nan,
    'price': df_product['prices'].values,
    'quantity': [random.randint(50, 1000) for j in range(len(df_product))],
    'image_url': df_product['img'].values,
    'discount': df_product['discounts'].values,
    'sold': df_product['countSales'].values,
    'rating': np.nan
})
df_product_backup

,product_id,name,description,price,quantity,image_url,discount,sold,rating
0,P0001,Dép Lê Thoáng Khí Đục Lỗ Bít Mũi Mùa Hè Đi Ra ...,NaN,95000,436,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/0.jpg,0.40,78,NaN
1,P0002,Dép Đế Xuồng Đế Dày Thời Trang Mùa Hè Cho Nữ Đ...,NaN,62000,997,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/1.jpg,0.87,6,NaN
2,P0003,SPICYG Darlingism Đầm Maxistyle Dài Thời Trang...,NaN,171071,867,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/2.jpg,0.58,22,NaN
3,P0004,ISARO Giày thể thao vải lưới thời trang 2024 d...,NaN,45000,753,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/3.jpg,0.50,31,NaN
4,P0005,"Dép nơ thời trang nữ, gót vuông 3 phân phù hợp...",NaN,120000,607,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/4.jpg,0.00,51,NaN
...,...,...,...,...,...,...,...,...,...
7155,P7156,Bộ tập pickleball áo polo PL chân váy xoè xếp ...,NaN,278000,751,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/715...,0.15,6,NaN
7156,P7157,Đồ tập gym yoga nữ set bộ áo crop ngắn tay kèm...,NaN,181000,933,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/715...,0.47,521,NaN
7157,P7158,mẫu đồ lam đi chùɑ #bộ bà lai gấm phối lụa-phá...,NaN,209000,546,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/716...,0.00,27,NaN
7158,P7159,[HCM] Đồ bộ lanh (tole) trung niên SHOP5SAO ta...,NaN,140600,458,C:/Users/ASUS/Desktop/T/ĐAN_KLTN/getImages/716...,0.24,6,NaN


In [22]:
df_product_backup.isna().sum()

product_id        0
name              0
description    7160
price             0
quantity          0
image_url         0
discount          0
sold              0
rating         7160
dtype: int64

In [23]:
df_product_backup.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\products.csv', index=False)

In [24]:
df_category['category_id'].value_counts()

category_id
SHIRT         1
DRESS         1
SET           1
PANTS         1
LONGSLEEVE    1
SKIRT         1
SHOES         1
TSHIRT        1
SHORTS        1
HAT           1
BODY          1
OUTWEAR       1
NOT_SURE      1
UNDERSHIRT    1
HOODIE        1
BLAZER        1
Name: count, dtype: int64

In [25]:
category_mapping = {category_id.lower(): category_id for category_id in df_category['category_id']}
category_mapping

{'shirt': 'SHIRT',
 'dress': 'DRESS',
 'set': 'SET',
 'pants': 'PANTS',
 'longsleeve': 'LONGSLEEVE',
 'skirt': 'SKIRT',
 'shoes': 'SHOES',
 'tshirt': 'TSHIRT',
 'shorts': 'SHORTS',
 'hat': 'HAT',
 'body': 'BODY',
 'outwear': 'OUTWEAR',
 'not_sure': 'NOT_SURE',
 'undershirt': 'UNDERSHIRT',
 'hoodie': 'HOODIE',
 'blazer': 'BLAZER'}

In [26]:
df = (
    df_product[['category', 'links']]
    .explode('category')
    .reset_index(drop=True)
)

df = df.merge(
    df_product_backup[['product_id']],
    left_index=True,
    right_index=True,
    how='left'
)

df['category'] = df['category'].str.strip("[]").str.replace("'", "").str.split(", ")
df = df.explode('category')

In [27]:
df_producthascategories = pd.DataFrame({
    'product_id': df['product_id'],
    'category_id': df['category'].str.lower().map(category_mapping).fillna('NOT_SURE')
})
df_producthascategories

,product_id,category_id
0,P0001,SHOES
0,P0001,HAT
1,P0002,SHOES
2,P0003,LONGSLEEVE
2,P0003,DRESS
...,...,...
7158,P7159,PANTS
7158,P7159,SET
7159,P7160,PANTS
7159,P7160,LONGSLEEVE


In [28]:
df_producthascategories.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\producthascategories.csv', index=False)

In [51]:
from datetime import datetime, timedelta

used_timestamps = set()
def generate_unique_timestamp():
    while True:
        ts = datetime.now() - timedelta(days=random.randint(0, 365), seconds=random.randint(0, 86400))
        ts = ts.replace(microsecond=random.randint(0, 999999))
        if ts not in used_timestamps:
            used_timestamps.add(ts)
            return ts

link_to_product_id = dict(zip(df_product['links'], df_product_backup['product_id']))

reviews = []
for idx, row in df_cmt.iterrows():
    product_id = link_to_product_id.get(row['Link'])
    if product_id:
        reviews.append({
            'customer_id': random.choice(df_customer['customer_id']),
            'product_id': product_id,
            'content': row['content_comment'],
            'rating': row['star_count'],
            'created_at': generate_unique_timestamp()
        })
df_review = pd.DataFrame(reviews)
df_review

,customer_id,product_id,content,rating,created_at
0,C032,P0001,"mang êm chân lắm, ai thích mang ôm chân lấy n...",5,2024-11-22 02:05:12.070343
1,C093,P0001,"Bền và chắc chắn, Khuyến khích trò chơi tưởng ...",5,2024-11-11 08:08:36.557479
2,C015,P0001,hàng tốt,5,2024-09-09 19:15:49.168221
3,C012,P0001,"Dép mang khg thoải mái,vì vô sát cổ chân quá,h...",3,2025-03-05 22:05:11.989998
4,C038,P0001,Nhẹ nhàng thoải mái độ cao vừa giúp chân sạch ...,5,2024-11-05 09:48:53.168130
...,...,...,...,...,...
126842,C086,P5898,"Hoàn hảo cho mùa hè, Họa tiết hoa phong cách, ...",5,2024-05-07 03:12:06.761547
126843,C066,P5898,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...",5,2025-03-13 17:53:11.619841
126844,C094,P5898,"Đa năng cho trang phục văn phòng, Họa tiết hoa...",5,2024-08-01 11:13:03.631849
126845,C029,P5898,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...",5,2024-09-18 09:08:35.605987


In [52]:
df_review = df_review.drop_duplicates(subset=['customer_id', 'product_id', 'created_at'])
df_review

,customer_id,product_id,content,rating,created_at
0,C032,P0001,"mang êm chân lắm, ai thích mang ôm chân lấy n...",5,2024-11-22 02:05:12.070343
1,C093,P0001,"Bền và chắc chắn, Khuyến khích trò chơi tưởng ...",5,2024-11-11 08:08:36.557479
2,C015,P0001,hàng tốt,5,2024-09-09 19:15:49.168221
3,C012,P0001,"Dép mang khg thoải mái,vì vô sát cổ chân quá,h...",3,2025-03-05 22:05:11.989998
4,C038,P0001,Nhẹ nhàng thoải mái độ cao vừa giúp chân sạch ...,5,2024-11-05 09:48:53.168130
...,...,...,...,...,...
126842,C086,P5898,"Hoàn hảo cho mùa hè, Họa tiết hoa phong cách, ...",5,2024-05-07 03:12:06.761547
126843,C066,P5898,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...",5,2025-03-13 17:53:11.619841
126844,C094,P5898,"Đa năng cho trang phục văn phòng, Họa tiết hoa...",5,2024-08-01 11:13:03.631849
126845,C029,P5898,"Cảm ơn chị đã tin dùng sản phẩm của shop, hy v...",5,2024-09-18 09:08:35.605987


In [53]:
df_review.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\reviews.csv', index=False)

In [32]:
carts = []
for _ in range(50):
    carts.append({
        'customer_id': random.choice(df_customer['customer_id']),
        'product_id': random.choice(df_product_backup['product_id']),
        'quantity': random.randint(1, 3)
    })

df_cart = pd.DataFrame(carts)
df_cart

,customer_id,product_id,quantity
0,C026,P1328,2
1,C018,P2189,1
2,C043,P1302,3
3,C091,P4358,2
4,C069,P5793,2
5,C070,P3661,2
6,C025,P0887,1
7,C078,P1992,1
8,C090,P0474,1
9,C069,P0065,2


In [33]:
df_cart.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\carts.csv', index=False)

In [34]:
customer_ids = df_customer['customer_id'].tolist()
status_ids = df_orderstatus['status_id'].tolist()
payment_ids = df_payment['payment_id'].tolist()

orders = []
for i in range(1, 51): 
    order_date = pd.Timestamp.now() - pd.Timedelta(days=random.randint(0, 365))
    order_status = random.choice(status_ids)
    orders.append({
        'order_id': f"O{i:04d}",
        'customer_id': random.choice(customer_ids),
        'order_date': order_date,
        'order_status': order_status,
        'shipped_date': order_date + pd.Timedelta(days=random.randint(1, 7)) if order_status == 'SHIPPED' else np.nan,
        'address': f'{random.randint(1, 250)} Đường số {random.randint(1, 10)}, Quận {random.randint(1, 11)}, TP.HCM',
        'note': None,
        'total_price': None,
        'payment': random.choice(payment_ids)
    })

df_orders = pd.DataFrame(orders)
df_orders

,order_id,customer_id,order_date,order_status,shipped_date,address,note,total_price,payment
0,O0001,C066,2024-11-03 01:32:04.616463,CANCELLED,NaT,"101 Đường số 8, Quận 11, TP.HCM",None,None,E_WALLET
1,O0002,C062,2025-03-28 01:32:04.616463,SHIPPED,2025-03-30 01:32:04.616463,"239 Đường số 3, Quận 11, TP.HCM",None,None,BANK_TRANSFER
2,O0003,C031,2024-06-30 01:32:04.616463,SHIPPED,2024-07-02 01:32:04.616463,"113 Đường số 5, Quận 10, TP.HCM",None,None,E_WALLET
3,O0004,C042,2024-12-25 01:32:04.616463,PENDING,NaT,"94 Đường số 9, Quận 11, TP.HCM",None,None,CREDIT_CARD
4,O0005,C095,2024-10-12 01:32:04.616463,CANCELLED,NaT,"21 Đường số 1, Quận 10, TP.HCM",None,None,BANK_TRANSFER
5,O0006,C061,2024-11-07 01:32:04.616463,PROCESSING,NaT,"88 Đường số 4, Quận 7, TP.HCM",None,None,COD
6,O0007,C082,2024-05-10 01:32:04.616463,PENDING,NaT,"203 Đường số 6, Quận 5, TP.HCM",None,None,BANK_TRANSFER
7,O0008,C077,2024-10-30 01:32:04.616463,SHIPPED,2024-11-05 01:32:04.616463,"131 Đường số 8, Quận 9, TP.HCM",None,None,CREDIT_CARD
8,O0009,C035,2025-03-27 01:32:04.616463,SHIPPED,2025-03-30 01:32:04.616463,"204 Đường số 1, Quận 10, TP.HCM",None,None,CREDIT_CARD
9,O0010,C051,2025-03-27 01:32:04.616463,SHIPPED,2025-04-01 01:32:04.616463,"5 Đường số 6, Quận 9, TP.HCM",None,None,E_WALLET


In [35]:
df_orders.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\orders.csv', index=False)

In [36]:
order_ids = df_orders['order_id'].tolist()
product_ids = df_product_backup['product_id'].tolist()

order_details = []
num_records = len(df_orders) * 2

for order_id in order_ids:
    order_details.append({
        'order_id': order_id,
        'product_id': random.choice(product_ids),
        'quantity': random.randint(1, 3),
        'unit_price': None,
        'discount': round(random.uniform(0, 0.5), 2)
    })

for i in range(int(len(order_ids) * 0.5)):
    order_details.append({
        'order_id': random.choice(order_ids),
        'product_id': random.choice(product_ids),
        'quantity': random.randint(1, 3),
        'unit_price': None,
        'discount': round(random.uniform(0, 0.5), 2)
    })
    
df_orderdetail = pd.DataFrame(order_details)
df_orderdetail

,order_id,product_id,quantity,unit_price,discount
0,O0001,P1266,3,None,0.18
1,O0002,P4971,1,None,0.00
2,O0003,P0330,3,None,0.02
3,O0004,P1202,1,None,0.50
4,O0005,P2546,2,None,0.21
...,...,...,...,...,...
70,O0018,P6198,1,None,0.04
71,O0043,P0236,2,None,0.11
72,O0030,P2055,1,None,0.38
73,O0001,P5106,3,None,0.41


In [37]:
df_orderdetail.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\orderdetail.csv', index=False)

In [38]:
admin_ids = df_admin['admin_id'].tolist()

activities = [
    "Tạo admin mới",
    "Xóa admin",
    "Cập nhật thông tin admin",
    "Xem báo cáo thu nhập",
    "Xuất báo cáo doanh thu",
    "Xem log hoạt động",
    "Khóa tài khoản người dùng",
    "Mở khóa tài khoản người dùng",
    "Cập nhật sản phẩm",
    "Xóa sản phẩm",
    "Thêm sản phẩm mới",
    "Xem thống kê đơn hàng",
    "Hủy đơn hàng",
    "Xác nhận đơn hàng"
]

activity_logs = []
num_records = 100 

for _ in range(num_records):
    activity_logs.append({
        'admin_id': random.choice(admin_ids),
        'activity': random.choice(activities),
        'activity_time': pd.Timestamp.now() - pd.Timedelta(days=random.randint(0, 365))
    })

df_activitylog = pd.DataFrame(activity_logs)
df_activitylog

,admin_id,activity,activity_time
0,C070,Khóa tài khoản người dùng,2024-06-02 01:32:04.678665
1,C089,Xác nhận đơn hàng,2024-07-10 01:32:04.678665
2,C063,Xem log hoạt động,2024-11-19 01:32:04.678665
3,C028,Xóa sản phẩm,2025-03-04 01:32:04.678665
4,C015,Xóa admin,2024-05-18 01:32:04.678665
...,...,...,...
95,C070,Xóa admin,2025-02-02 01:32:04.680767
96,C020,Tạo admin mới,2024-08-17 01:32:04.680767
97,C032,Cập nhật sản phẩm,2024-08-14 01:32:04.680767
98,C080,Xác nhận đơn hàng,2025-01-24 01:32:04.680767


In [39]:
df_activitylog.to_csv(r'C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup\activitylogs.csv', index=False)

# LOAD DATA TO DATABASE

In [54]:
# !pip install sqlalchemy
# !pip install pymysql

In [55]:
import pymysql
import os

In [56]:
folder_path = r"C:\Users\ASUS\Desktop\T\ĐAN_KLTN\eCommerce_backup"

In [57]:
csv_table_mapping = {
    'payments.csv': 'payment',
    'categories.csv': 'category',
    'permissions.csv': 'permission',
    'orderstatuses.csv': 'orderstatus',
    'admins.csv': 'admin',
    'adminhaspermissions.csv': 'adminhaspermissions',
    'customers.csv': 'customer',
    'products.csv': 'product',
    'producthascategories.csv': 'producthascategories',
    'carts.csv': 'cart',
    'reviews.csv': 'review',
    'orders.csv': 'orders',
    'orderdetail.csv': 'orderdetail'
}

In [58]:
csv_table_mapping = {
    'reviews.csv': 'review'
}

In [59]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Abcxyz@123",
    database="eCommerce"
)
cursor = conn.cursor()

In [60]:
def upload_csv_to_mysql_folder():
    for file_name, table_name in csv_table_mapping.items():
        file_path = os.path.join(folder_path, file_name)

        if not os.path.exists(file_path):
            print(f"⚠️ Không tìm thấy file: {file_path}")
            continue

        print(f"\n📤 Đang xử lý: {file_name} → {table_name}")
        try:
            df = pd.read_csv(file_path)
            if df.empty:
                print(f"⚠️ Bỏ qua vì file rỗng: {file_name}")
                continue

            columns = df.columns.tolist()
            col_names = ", ".join(f"`{col}`" for col in columns)
            placeholders = ", ".join(["%s"] * len(columns))
            insert_query = f"INSERT INTO `{table_name}` ({col_names}) VALUES ({placeholders})"

            for _, row in df.iterrows():
                values = tuple(None if pd.isna(row[col]) else row[col] for col in columns)
                cursor.execute(insert_query, values)

            conn.commit()
            print(f"✅ Đã chèn vào `{table_name}` ({len(df)} dòng) và kích hoạt trigger nếu có.")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý {file_name}: {e}")
            conn.rollback()

In [61]:
upload_csv_to_mysql_folder()


📤 Đang xử lý: reviews.csv → review
✅ Đã chèn vào `review` (126847 dòng) và kích hoạt trigger nếu có.


In [62]:
cursor.close()
conn.close()